In [1]:
%load_ext lab_black
%cd ..

/mnt/h/hev/log-analytics


In [2]:
import argparse
import math
import multiprocessing
import sys
from datetime import datetime
from pathlib import Path
from pprint import pformat

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_optimizer
import yaml
from easydict import EasyDict
from pytorch_transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    DistilBertForSequenceClassification,
    DistilBertTokenizer,
    RobertaForSequenceClassification,
    RobertaTokenizer,
)
from sklearn.model_selection import StratifiedKFold
from torch.optim import Adam, AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from transformers import (
    AlbertForSequenceClassification,
    AlbertTokenizer,
    DebertaForSequenceClassification,
    DebertaTokenizer,
    SqueezeBertTokenizer,
    SqueezeBertForSequenceClassification,
    XLNetTokenizer,
    XLNetForSequenceClassification,
)

from datasets import load_test_data, load_train_data, MyDataset, load_train_total_data
from utils import SAM, AverageMeter, CustomLogger, FocalLoss, seed_everything

from main import MyTrainer
from collections import defaultdict
import matplotlib.pyplot as plt
import random

In [3]:
def get_dist(deck, feat, topk):
    dist = torch.norm(deck - feat[None], dim=1, p=None)
    knn = dist.topk(topk, largest=False)
    values, indices = knn
    return values, indices

In [4]:
with open("config/distilbert-base-uncased.yaml", "r") as f:
    C = EasyDict(yaml.load(f, yaml.FullLoader))
    C.result_dir = Path(C.result_dir)
    C.dataset.dir = Path(C.dataset.dir)
    seed_everything(C.seed, deterministic=False)

In [5]:
C

{'model': {'name': 'distilbert-base-uncased'},
 'comment': None,
 'result_dir': PosixPath('results/distilbert-base-uncased'),
 'debug': False,
 'seed': 20210425,
 'train': {'SAM': False,
  'folds': [1],
  'checkpoints': [None],
  'loss': {'name': 'focal', 'gamma': 2},
  'optimizer': {'name': 'AdamW'},
  'finetune': {'do': True, 'step1_epochs': 2, 'step2_epochs': 4},
  'max_epochs': 10,
  'lr': 1e-05,
  'scheduler': {'name': 'ReduceLROnPlateau',
   'params': {'factor': 0.5, 'patience': 3, 'verbose': True}}},
 'dataset': {'dir': PosixPath('data/ori'), 'batch_size': 30, 'num_workers': 8}}

In [6]:
trainer = MyTrainer(C, 1, "results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr1e-05_1.pth")

Load pretrained results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr1e-05_1.pth


In [7]:
model = trainer.model
model.eval()
torch.set_grad_enabled(False)

In [8]:
activation = []


def hook(model, input, output):
    activation.append(output.detach().cpu())

In [9]:
model.pre_classifier.register_forward_hook(hook)

In [10]:
dl = load_train_total_data(C.dataset.dir, trainer.tokenizer, 100, 6)

## tdeck 만들기 (모든 train 데이터에 대한 집합)

In [ ]:
activation = []
deck = {
    'feat': [],
    'tlevel': [],
    'fclevel': [],
    'otext': [],
}
with tqdm(total=len(dl.dataset), ncols=100, file=sys.stdout) as t:
    for id, text, tlevel, otext in dl:
        pred = model(text.cuda(non_blocking=True))[0].cpu()
        deck['feat'].append(pred)
        deck['tlevel'].append(tlevel)
        deck['fclevel'].append(pred.argmax(dim=1))
        deck['otext'].extend(otext)
        
        t.update(len(id))

 88%|███████████████████████████████████████████████▎      | 414700/472972 [22:08<03:04, 315.61it/s]

## sdeck 만들기 (모든 test 데이터에 대한 집합)

In [11]:
# sfeats 저장
activation = []
sfcfeats = []
with tqdm(total=len(trainer.dl_test.dataset), ncols=100, file=sys.stdout) as t:
    for _, text, otext in trainer.dl_test:
        sfcfeat = model(text.cuda(non_blocking=True))[0].cpu()
        sfcfeats.append(sfcfeat)
        t.update(len(text))

100%|██████████████████████████████████████████████████| 1418916/1418916 [1:11:04<00:00, 332.74it/s]


In [12]:
sfeats = torch.cat(activation)
sfcfeats = torch.cat(sfcfeats)

In [13]:
np.savez_compressed(
    "results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr1e-05_1-sfeats.npz", sfeats=sfeats, sfcfeats=sfcfeats
)

## dist값, KNN level, FC level 을 저장

여기서 reload

In [14]:
data = np.load("results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr1e-05_1-sfeats.npz")
sfeats = data["sfeats"]
sfcfeats = data["sfcfeats"]

In [15]:
sfeats = torch.from_numpy(sfeats)
sfcfeats = torch.from_numpy(sfcfeats)

In [16]:
sfeats.shape

torch.Size([1418916, 768])

In [19]:
sfeats_ = sfeats.cuda()

In [20]:
deck = torch.load("results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr1e-05_1-deck.pth")

In [21]:
deck["feat_"] = deck["feat"].cuda()

In [22]:
deck.keys()

dict_keys(['feat', 'tlevel', 'fclevel', 'feat_'])

In [21]:
# dist를 구함
dists, fclevels, tlevels, otexts = [], [], []
with tqdm(total=len(trainer.dl_test.dataset), ncols=100, file=sys.stdout) as t:
    for sfeat_, sfcfeat in zip(sfeats_, sfcfeats):
        dist_, index_ = get_dist(deck["feat_"], sfeat_, 8)
        dists.append(dist_.cpu())
        tlevels.append(deck["tlevel"][index_[0]].item())
        fclevels.append(sfcfeat)

        t.update()

100%|██████████████████████████████████████████████████| 1418916/1418916 [3:14:52<00:00, 121.36it/s]


In [23]:
dists[0], fclevels[0], tlevels[0]

(tensor([0., 0., 0., 0., 0., 0., 0., 0.]),
 tensor([ 4.6420, -3.8289, -8.8472, -3.1232, -9.0210, -4.3306, -8.9362]),
 0)

In [24]:
dists = torch.stack(dists)

In [25]:
fclevels = torch.stack(fclevels)

In [30]:
tlevels = torch.tensor(tlevels, dtype=torch.long)

In [41]:
sfeats.shape, dists.shape

(torch.Size([1418916, 768]), torch.Size([1418916, 8]))

In [45]:
np.savez_compressed(
    "results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr1e-05_1-dist.npz",
    sfeats=sfeats.numpy(),
    dists=dists.numpy(),
    tlevels=tlevels.numpy(),
    fclevels=fclevels.numpy(),
)